In [2]:
%load_ext autoreload
%autoreload 2

from utils.imports import *
from utils.plotting import *
from utils.config import DEFAULT_CONFIG as cfg
from utils.binary import get_blackhole_binary
from utils.star import evolve_star, relax_sph_realization
from amuse.ext.sink import new_sink_particles


/home/boydjo/miniforge3/envs/amuse-env/lib/python3.12/site-packages/amuse/support/options.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [22]:

# Read the simulation file
data = read_set_from_file("Lady_Tramp_50_50_0.hdf5") #"Lady_Tramp_50_50_pi.hdf5","Lady_Tramp_70_30_0.hdf5", "Lady_Tramp_70_30_pi.hdf5", "Lady_Tramp_40_60_0.hdf5", "Lady_Tramp_40_60_pi.hdf5"

binary = data['binary']
gas = data['gas']

# Mass of one gas particle
m_particle = gas[0].mass  # 0.00016 MSun in your case

# Initial masses
initial_masses = {
    "primary": 50 | units.MSun,
    "secondary": 50 | units.MSun,
}

# Dictionary to store accreted masses and number of particles
accreted_mass = {}
accreted_particles = {}

for bh in binary:
    delta_m = bh.mass - initial_masses[bh.name]
    accreted_mass[bh.name] = delta_m
    accreted_particles[bh.name] = (delta_m / m_particle) # unitless number
    print(f"{bh.name}: ΔM = {delta_m.in_(units.MSun)}, accreted particles ≈ {accreted_particles[bh.name]:.0f}")


primary: ΔM = 0.0033MSun, accreted particles ≈ 21
secondary: ΔM = 0.79MSun, accreted particles ≈ 4954


In [35]:
from amuse.io import read_set_from_file
from amuse.units import units
import matplotlib.pyplot as plt

# List of simulation files
sim_files = [
    "Lady_Tramp_50_50_0.hdf5",
    "Lady_Tramp_50_50_pi.hdf5",
    "Lady_Tramp_70_30_0.hdf5",
    "Lady_Tramp_70_30_pi.hdf5",
    "Lady_Tramp_40_60_0.hdf5",
    "Lady_Tramp_40_60_pi.hdf5"
]

for sim_file in sim_files:
    # Extract numbers from filename
    parts = sim_file.split("_")
    num1 = int(parts[2])
    num2 = int(parts[3].split(".")[0])

    # Flip only the 70-30 cases
    if num1 == 70 and num2 == 30:
        primary_mass_init = num2 | units.MSun  # 30
        secondary_mass_init = num1 | units.MSun  # 70
    else:
        primary_mass_init = num1 | units.MSun
        secondary_mass_init = num2 | units.MSun

    initial_masses = {
        "primary": primary_mass_init,
        "secondary": secondary_mass_init,
    }

    # Read the simulation
    data = read_set_from_file(sim_file)
    binary = data['binary']
    gas = data['gas']

    m_particle = gas[0].mass  # Mass of one gas particle

    accreted_particles = []

    for bh in binary:
        delta_m = bh.mass - initial_masses[bh.name]
        n_particles = (delta_m / m_particle)
        accreted_particles.append(n_particles)
        print(f"{sim_file} - {bh.name}: ΔM = {delta_m.in_(units.MSun)}, accreted particles ≈ {n_particles:.0f}")

    # Plot histogram
    plt.figure()
    plt.bar(["Primary", "Secondary"], accreted_particles, color=["blue", "red"], width=0.3)
    plt.ylabel("Number of Accreted Particles")
    plt.title(f"Accreted Particles - {sim_file}")

    # Save the figure as PNG (does not touch the .hdf5 file)
    plt.savefig(sim_file.replace(".hdf5", "_accreted_particles.png"))
    plt.close()



Lady_Tramp_50_50_0.hdf5 - primary: ΔM = 0.0033MSun, accreted particles ≈ 21
Lady_Tramp_50_50_0.hdf5 - secondary: ΔM = 0.79MSun, accreted particles ≈ 4954
Lady_Tramp_50_50_pi.hdf5 - primary: ΔM = 0.79MSun, accreted particles ≈ 4956
Lady_Tramp_50_50_pi.hdf5 - secondary: ΔM = 0.0033MSun, accreted particles ≈ 21
Lady_Tramp_70_30_0.hdf5 - primary: ΔM = 0.0MSun, accreted particles ≈ 0
Lady_Tramp_70_30_0.hdf5 - secondary: ΔM = 0.00032MSun, accreted particles ≈ 2
Lady_Tramp_70_30_pi.hdf5 - primary: ΔM = 0.0MSun, accreted particles ≈ 0
Lady_Tramp_70_30_pi.hdf5 - secondary: ΔM = 0.79MSun, accreted particles ≈ 4978
Lady_Tramp_40_60_0.hdf5 - primary: ΔM = 0.0067MSun, accreted particles ≈ 42
Lady_Tramp_40_60_0.hdf5 - secondary: ΔM = 0.66MSun, accreted particles ≈ 4171
Lady_Tramp_40_60_pi.hdf5 - primary: ΔM = 0.00064MSun, accreted particles ≈ 4
Lady_Tramp_40_60_pi.hdf5 - secondary: ΔM = 0.75MSun, accreted particles ≈ 4742
